## Case Study - Operations analytics

You own a call center with 1000 resources. Each of these resources have gone through a different set of trainings. These training combinations are A,B,C,D,E and F. For instance set A is the population who have gone through training 1,2,3,4,5 and 6. Similarly, set B has gone through some other set of trainings. The distribution of these callers are

| Training | No. of Resources|
| :--: | :--:|
| A | 200 |
| B | 200 |
| C | 100 |
| D | 100 |
| E | 250 |
| F | 150 |

Also, the calls you get on a daily basis have different types of queries. This can again be classified into 4 categories. Let’s call them X,Y,Z and W. Each of these category can be identified through an IVR and you can try to allot any caller to these customers. On a daily basis you get following distribution of calls:

| Category | % of calls|
| :--: | :--:|
| X | 20% |
| Y | 30% |
| Z | 15% |
| W | 35% |

The number of calls might vary with days but remains almost constant at this ratio. With every set of trainings, you get a different type of skill set which enables a caller to resolve varied type of calls in different time. For instance, a set A agent can resolve a call X in 10 minutes. Following is a grid, which you can refer to check the resolution time for each combination:

| Time(min) | A | B | C | D | E | F |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| **X** | 10 | 11 | 10 | 1 | 12 | 5 |
| **Y** | 8 | 4 | 9 | 12 | 7 | 5 |
| **Z** | 2 | 11 | 4 | 11 | 6 | 11 |
| **W** | 3 | 12 | 7 | 14 | 4 | 11 |

The objective is to complete the work of all the callers in the least aggregate time. Which will allow one to train them on different skills and therefore decrease the resolution time dynamically.

The case study has been taken from: https://www.analyticsvidhya.com/blog/2016/06/operations-analytics-case-study-level-hard/

## Solution

**Sets:**

$r \in \text{Training} = \{\text{A},\text{B},\text{C},\text{D},\text{E},\text{F}\}$

$c \in \text{Category} = \{\text{X},\text{Y},\text{Z},\text{W}\}$

**Parameters:**

$\text{Resources}_{r} \in \mathbb{I}^+$: Number of resources with training $r$.

$\text{Calls}_{c} \in \mathbb{I}^+$: Number of calls of category $c$.

$\text{Time}_{r,c} \in \mathbb{I}^+$: Time (in min) taken by a resource with training $r$ to resolve a call of category $c$.


**Variables:**

$\text{X}_{r,c} \in \mathbb{I}^+$: No. of resources with training $r$ attending call of category $c$.

**Objective Function:**

\begin{equation}
\text{Min} \sum_{r \in \text{Training}} \left(\sum_{c \in \text{Category}} \text{X}_{r,c}*\text{Time}_{r,c}\right)
\tag{0}
\end{equation}

**Constraints:**

\begin{equation}
\sum_{r \in \text{Training}} \text{X}_{r,c} \le \text{Resources}_{r} \quad \forall c \in \text{Category}
\tag{1}
\end{equation}

\begin{equation}
\sum_{c \in \text{Category}} \text{X}_{r,c} = \text{Calls}_{c} \quad \forall r \in \text{Training}
\tag{1}
\end{equation}

In [3]:
import pyomo.environ as pyomo
import pandas as pd

In [4]:
model = pyomo.ConcreteModel()

# Sets
model.training = pyomo.RangeSet(6)
model.category = pyomo.RangeSet(4)

# Variables
model.x = pyomo.Var(model.training, model.category, domain = pyomo.NonNegativeIntegers)

# Parameters
model.resources = pyomo.Param(model.training, initialize = {1:200, 2:200, 3:100, 4:100, 5:250, 6:150})

model.calls = pyomo.Param(model.category, initialize = {1:200, 2:300, 3:150, 4:350})

model.time = pyomo.Param(model.training, model.category, initialize = {(1,1):10, (2,1):11, (3,1):10, (4,1):1, (5,1):12, (6,1):5,
                                                                       (1,2):8, (2,2):4, (3,2):9, (4,2):12, (5,2):7, (6,2):5,
                                                                       (1,3):2, (2,3):11, (3,3):4, (4,3):11, (5,3):6, (6,3):11,
                                                                       (1,4):3, (2,4):12, (3,4):7, (4,4):14, (5,4):4, (6,4):11,})

# Objective Function
def obj(model):
  value = sum(sum(model.x[r,c]*model.time[r,c] for c in model.category) for r in model.training)
  return value
model.obj = pyomo.Objective(rule = obj, sense = pyomo.minimize)

# Constraints
def rule1(model, r):
  return sum(model.x[r,c] for c in model.category) <= model.resources[r]
model.rule1 = pyomo.Constraint(model.training, rule = rule1)

def rule2(model, c):
  return sum(model.x[r,c] for r in model.training) == model.calls[c]
model.rule2 = pyomo.Constraint(model.category, rule = rule2)

# Solver options
results = pyomo.SolverFactory('glpk').solve(model)

# Printing results
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3750.0
  Upper bound: 3750.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 25
  Number of nonzeros: 49
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.07361555099487305
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [9]:
print('\n RESULTS \n')
print('The minimum time required to resolve all the calls is ',model.obj(),'min')

df = pd.DataFrame(columns=model.training, index=model.category)
for r in model.training:
    for c in model.category:
        df[r][c] = model.x[r,c]()
        
df.columns = ['A','B','C','D','E','F']
df.index = ['X','Y','Z','W']

df.loc['Total',:]= df.sum(axis=0)
df.loc[:,'Total']= df.sum(axis=1)
print('\n Resource Allocation Table \n')
df


 RESULTS 

The minimum time required to resolve all the calls is  3750.0 min

 Resource Allocation Table 



,A,B,C,D,E,F,Total
X,0.0,0.0,0.0,100.0,0.0,100.0,200.0
Y,0.0,200.0,0.0,0.0,50.0,50.0,300.0
Z,50.0,0.0,100.0,0.0,0.0,0.0,150.0
W,150.0,0.0,0.0,0.0,200.0,0.0,350.0
Total,200.0,200.0,100.0,100.0,250.0,150.0,1000.0
